In [9]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

In [3]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [12]:
dataset_dir = '../input/ann-challenge-500-balanced/balanced_dataset/training'
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

In [7]:
batch_size = 256
img_height = 256
img_width = 256
input_shape = (256,256,3)
epochs = 100

In [10]:
idg = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=5,
    height_shift_range=5,
    rotation_range=30,
    fill_mode='constant',
    cval=0,
    validation_split=0.2) 

In [13]:
train_data_generator = idg.flow_from_directory(
                  directory=dataset_dir, target_size=(img_width, img_height),
                  batch_size=batch_size, seed=seed, subset="training")
  
valid_data_generator = idg.flow_from_directory(
                  directory=dataset_dir, target_size=(img_width, img_height), 
                  batch_size=batch_size, seed=seed, subset="validation")

In [14]:
# Download and plot the VGG16 model
supernet = tfk.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3)
)
supernet.summary()

In [15]:
# Use the supernet as feature extractor
supernet.trainable = False

inputs = tfk.Input(shape=(img_height,img_width,3))
x = preprocess_input(inputs)
x = tfkl.Resizing(256,256, interpolation='bicubic')(x)
x = supernet(x)
x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
x = tfkl.Dense(
    256, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
outputs = tfkl.Dense(
    14, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


# Connect input and output through the Model class
fine_tuning_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
fine_tuning_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
fine_tuning_model.summary()

In [16]:
# Set all VGG layers to True
fine_tuning_model.get_layer('vgg16').trainable = True
for i, layer in enumerate(fine_tuning_model.get_layer('vgg16').layers):
   print(i, layer.name, layer.trainable)

In [18]:
# Freeze first N layers, e.g., until 14th
for i, layer in enumerate(fine_tuning_model.get_layer('vgg16').layers[:14]):
  layer.trainable=False
for i, layer in enumerate(fine_tuning_model.get_layer('vgg16').layers):
   print(i, layer.name, layer.trainable)
fine_tuning_model.summary()

In [20]:
# Train the model
history = fine_tuning_model.fit(
    train_data_generator,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = valid_data_generator,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

In [21]:
fine_tuning_model.save('fine_tuning_Model')

In [22]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history['loss'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [23]:
max(history['val_accuracy'])